In [1]:
import csv
import numpy as np 

In [2]:
###  Load data

def read_csv(file):
    with open(file, newline='') as f:
        reader = csv.reader(f)
        line_count = 0
        rows = []
        for row in reader:
            if line_count == 0:
                titles = row
            else:
                rows.append(row)
            line_count += 1
    rows_int = np.array([[int(r) for r in row] for row in rows])
    return titles, rows_int
        

titles, rows_train = read_csv('data/studentsdigits-train.csv')
assert titles[-1] == 'Digit' and len(titles) == 9, 'Not train set'
X_train = rows_train[:,0:len(titles)-1]
Y_train = rows_train[:,-1]
print(X_train.shape)
print(Y_train.shape)

titles, rows_test = read_csv('data/studentsdigits-test.csv')
assert len(titles) == 8, 'Not test set'
X_test = rows_test
print(X_test.shape)

(3747, 8)
(3747,)
(3747, 8)


In [3]:
import torch
from torch import nn
from sklearn.model_selection import KFold
from collections import OrderedDict
from sklearn.metrics import accuracy_score
from tqdm import tqdm, trange

In [4]:
# Build the MLP model


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        torch.manual_seed(0)  # reproducibility

        # self.layers = nn.ModuleDict(
        #     OrderedDict(
        #         [
        #             ("fc_1", nn.Linear(in_features=8, out_features=32)),
        #             ("relu_1", nn.ReLU()),
        #             ("fc_2", nn.Linear(in_features=32, out_features=16)),
        #             ("relu_2", nn.ReLU()),
        #             ("fc_3", nn.Linear(in_features=16, out_features=10)),
        #             ("relu_3", nn.ReLU()),
        #         ]
        #     )
        # )
        # Maybe a little bit deepere rnn
        self.rnn = nn.RNN(input_size=2, hidden_size=16, num_layers=3, batch_first=True)
        self.fc = nn.Linear(in_features=16, out_features=10)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.lossFunction = nn.CrossEntropyLoss()

    def forward(self, x):
        # for layer in self.layers.values():
        #     x = layer(x)
        # return x

        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])
        return x

    def run(self, X_train, Y_train):
        kf = KFold(n_splits=10, shuffle=False)
        # k-fold cross validation
        for fold, (train_index, test_index) in enumerate(
            tqdm(kf.split(X_train), total=kf.get_n_splits())
        ):
            self.train()
            x_train_fold, x_evaluate_fold = X_train[train_index], X_train[test_index]
            y_train_fold, y_evaluate_fold = Y_train[train_index], Y_train[test_index]

            x_train_foldTensor = torch.tensor(x_train_fold, dtype=torch.float32).view(
                -1, 4, 2
            )
            # x_train_foldTensor = torch.tensor(x_train_fold, dtype=torch.float32)
            y_train_foldTensor = torch.tensor(y_train_fold, dtype=torch.long)
            x_evaluate_fold = torch.tensor(x_evaluate_fold, dtype=torch.float32).view(
                -1, 4, 2
            )
            # x_evaluate_fold = torch.tensor(x_evaluate_fold, dtype=torch.float32)
            y_evaluate_fold = torch.tensor(y_evaluate_fold, dtype=torch.long)

            # print(f'Shape of x_train_foldTensor is: {x_train_foldTensor.shape}')
            # print(f'Value of of one entry in x_train_foldTensor is: {x_train_foldTensor[0]}')
            for epoch in trange(500, desc=f"Fold {fold+1}", leave=False):
                self.optimizer.zero_grad()
                fold_pred = self.forward(x_train_foldTensor)
                # fold_true = torch.LongTensor(np.array(y_train_fold))
                loss = self.lossFunction(fold_pred, y_train_foldTensor)
                loss.backward()
                self.optimizer.step()

            # Evaluate using this fold
            self.eval()
            fold_evaluate_pred = self.forward(x_evaluate_fold)
            # fold_evaluate_true = torch.LongTensor(np.array(y_evaluate_fold))

            # Metrics
            acc = accuracy_score(y_evaluate_fold, fold_evaluate_pred.argmax(dim=1))
            lossItem = loss.item()
            print(f"Epoch: {epoch}, Loss: {lossItem}, Accuracy: {acc}")

In [5]:
# Train model
model = Model()
model.run(X_train, Y_train)

 10%|█         | 1/10 [00:03<00:31,  3.51s/it]

Epoch: 499, Loss: 0.30239948630332947, Accuracy: 0.0


 20%|██        | 2/10 [00:07<00:28,  3.54s/it]

Epoch: 499, Loss: 0.09637830406427383, Accuracy: 0.112


 30%|███       | 3/10 [00:10<00:24,  3.52s/it]

...

Epoch: 499, Loss: 0.009206902235746384, Accuracy: 0.9037433155080213


 90%|█████████ | 9/10 [00:31<00:03,  3.51s/it]

Epoch: 499, Loss: 0.01045581791549921, Accuracy: 0.9090909090909091


100%|██████████| 10/10 [00:35<00:00,  3.52s/it]

Epoch: 499, Loss: 0.007662969175726175, Accuracy: 0.8796791443850267


In [6]:
### You code

### The predictions of test set: Y_test
Y_test = []

In [7]:
### Save prediction results
assert len(Y_test) == len(X_test), 'sizes dont match'
with open('upload_predictions.txt', 'w') as fp:
    fp.write('\n'.join(str(y) for y in Y_test))
print('SAVED')

AssertionError: sizes dont match